# Data Filtering & Selection

### Loading Libraries

In [60]:
# ZipFiles
import zipfile

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import polars as pl
import polars.selectors as cs

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# XGBoost
import xgboost as xgb

# Scikit-Learn
from sklearn import decomposition

# Date & Time
from datetime import datetime
from datetime import timedelta

## Loading Data

In [2]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [3]:
raw = pl.read_csv(path, null_values=['NA'])

In [4]:
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany')                    
                          .str.contains('Automatic')
                          .fill_null('Automatic'),
                          num_gears=pl.col('trany')
                          .str.extract(r'(\d+)')
                          .cast(pl.UInt8)
                          .fill_null(6))
           )

In [5]:
autos = tweak_auto(raw)

In [6]:
print(autos)

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

### Filtering with Expressions

In [7]:
print(autos
      .filter(pl.col('city08') > 40)
     )

shape: (1_325, 15)
┌──────┬────────┬───────────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make   ┆ model     ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---    ┆ ---       ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat    ┆ cat       ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪════════╪═══════════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
│ 1994 ┆ Geo    ┆ Metro XFI ┆ 1.0   ┆ … ┆ 52        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆        ┆           ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 2000 ┆ Honda  ┆ Insight   ┆ 1.0   ┆ … ┆ 61        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆        ┆           ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 2000 ┆ Nissan ┆ Altra EV  ┆ null  ┆ … ┆ 91        ┆ 2013-01-01   ┆ Automatic    ┆ 6         │
│      ┆        ┆    

In [8]:
print(autos
      .filter(pl.col('city08') > 40)
      .filter(pl.col('make') == 'Toyota')
     )

shape: (155, 15)
┌──────┬────────┬───────────────┬───────┬───┬───────────┬───────────────┬──────────────┬───────────┐
│ year ┆ make   ┆ model         ┆ displ ┆ … ┆ highway08 ┆ createdOn     ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---    ┆ ---           ┆ ---   ┆   ┆ ---       ┆ ---           ┆ ---          ┆ ---       │
│ i16  ┆ cat    ┆ cat           ┆ f32   ┆   ┆ u8        ┆ datetime[μs]  ┆ str          ┆ u8        │
╞══════╪════════╪═══════════════╪═══════╪═══╪═══════════╪═══════════════╪══════════════╪═══════════╡
│ 2000 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ Automatic    ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ Prius         ┆ 1.5   ┆ … ┆ 41        ┆ 2013-01-01    ┆ true         ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ 

In [9]:
# Placing Multiple Conditions within Filter
print(autos
      .filter(pl.col('city08') > 40, pl.col('make') == 'Toyota')
     )

shape: (155, 15)
┌──────┬────────┬───────────────┬───────┬───┬───────────┬───────────────┬──────────────┬───────────┐
│ year ┆ make   ┆ model         ┆ displ ┆ … ┆ highway08 ┆ createdOn     ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---    ┆ ---           ┆ ---   ┆   ┆ ---       ┆ ---           ┆ ---          ┆ ---       │
│ i16  ┆ cat    ┆ cat           ┆ f32   ┆   ┆ u8        ┆ datetime[μs]  ┆ str          ┆ u8        │
╞══════╪════════╪═══════════════╪═══════╪═══╪═══════════╪═══════════════╪══════════════╪═══════════╡
│ 2000 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ Automatic    ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ Prius         ┆ 1.5   ┆ … ┆ 41        ┆ 2013-01-01    ┆ true         ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ 

In [10]:
# Using Boolean Operators
print(autos
      .filter((pl.col('make') == 'Toyota') & (pl.col('city08') > 40))
     )

shape: (155, 15)
┌──────┬────────┬───────────────┬───────┬───┬───────────┬───────────────┬──────────────┬───────────┐
│ year ┆ make   ┆ model         ┆ displ ┆ … ┆ highway08 ┆ createdOn     ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---    ┆ ---           ┆ ---   ┆   ┆ ---       ┆ ---           ┆ ---          ┆ ---       │
│ i16  ┆ cat    ┆ cat           ┆ f32   ┆   ┆ u8        ┆ datetime[μs]  ┆ str          ┆ u8        │
╞══════╪════════╪═══════════════╪═══════╪═══╪═══════════╪═══════════════╪══════════════╪═══════════╡
│ 2000 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ Automatic    ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ Prius         ┆ 1.5   ┆ … ┆ 41        ┆ 2013-01-01    ┆ true         ┆ 6         │
│      ┆        ┆               ┆       ┆   ┆           ┆ 00:00:00      ┆              ┆           │
│ 2001 ┆ Toyota ┆ RAV4 EV       ┆ null  ┆ … ┆ 64        ┆ 2013-01-01    ┆ 

### Filtering with Keywords

In [11]:
print(autos
      .filter(make='Ford')
     )

shape: (3_835, 15)
┌──────┬──────┬─────────────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make ┆ model       ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---         ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat  ┆ cat         ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪══════╪═════════════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
│ 1993 ┆ Ford ┆ Taurus      ┆ 3.0   ┆ … ┆ 27        ┆ 2013-01-01   ┆ true         ┆ 4         │
│      ┆      ┆             ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1993 ┆ Ford ┆ Taurus SHO  ┆ 3.0   ┆ … ┆ 24        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆      ┆             ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1993 ┆ Ford ┆ Taurus      ┆ 3.8   ┆ … ┆ 25        ┆ 2013-01-01   ┆ true         ┆ 4         │
│      ┆      ┆      

### Filtering with Dates

In [13]:
mar2018 = datetime(2018, 3, 1)

In [15]:
print(autos
      .filter(pl.col('createdOn') >= mar2018)
      .sort('createdOn')
     )

shape: (8_694, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 2019 ┆ Genesis    ┆ G70 AWD      ┆ 3.3   ┆ … ┆ 25        ┆ 2018-03-02  ┆ true        ┆ 8         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 2019 ┆ Genesis    ┆ G70 RWD      ┆ 3.3   ┆ … ┆ 26        ┆ 2018-03-02  ┆ true        ┆ 8         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00   

In [17]:
print(autos
      .filter(pl.col('city08').is_between(40, 50))
      .sort('city08')
      .select(['year', 'make', 'model', 'VClass', 'city08'])
     )

shape: (328, 5)
┌──────┬────────┬────────────────┬──────────────┬────────┐
│ year ┆ make   ┆ model          ┆ VClass       ┆ city08 │
│ ---  ┆ ---    ┆ ---            ┆ ---          ┆ ---    │
│ i16  ┆ cat    ┆ cat            ┆ cat          ┆ u8     │
╞══════╪════════╪════════════════╪══════════════╪════════╡
│ 1985 ┆ Honda  ┆ Civic CRX HF   ┆ Two Seaters  ┆ 40     │
│ 2003 ┆ Honda  ┆ Civic Hybrid   ┆ Compact Cars ┆ 40     │
│ 2006 ┆ Honda  ┆ Civic Hybrid   ┆ Compact Cars ┆ 40     │
│ 2007 ┆ Honda  ┆ Civic Hybrid   ┆ Compact Cars ┆ 40     │
│ 2008 ┆ Honda  ┆ Civic Hybrid   ┆ Compact Cars ┆ 40     │
│ …    ┆ …      ┆ …              ┆ …            ┆ …      │
│ 2024 ┆ Toyota ┆ Corolla Hybrid ┆ Compact Cars ┆ 50     │
│ 2024 ┆ Toyota ┆ Prius Prime    ┆ Midsize Cars ┆ 50     │
│ 2025 ┆ Honda  ┆ Civic 4Dr      ┆ Midsize Cars ┆ 50     │
│ 2025 ┆ Toyota ┆ Corolla Hybrid ┆ Compact Cars ┆ 50     │
│ 2025 ┆ Honda  ┆ Civic 5Dr      ┆ Large Cars   ┆ 50     │
└──────┴────────┴────────────────┴──────

In [19]:
mar2013 = mar2018 - timedelta(days=5*365)

In [20]:
print(autos
      .filter(pl.col('createdOn').is_between(mar2013, mar2018))
      .sort('createdOn')
     )

shape: (5_344, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 2014 ┆ Lexus      ┆ GS 350       ┆ 3.5   ┆ … ┆ 29        ┆ 2013-10-29  ┆ true        ┆ 8         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 2014 ┆ Nissan     ┆ Frontier 2WD ┆ 2.5   ┆ … ┆ 23        ┆ 2013-10-29  ┆ true        ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00   

### Advanced Filtering Techniques

In [21]:
def standardize(col):
    return (col - col.mean()) / col.std()

In [22]:
print(autos
      .filter(standardize(pl.col('city08')) > 3)
      .select(['year', 'make', 'model', 'VClass', 'city08'])
     )

shape: (949, 5)
┌──────┬───────────────┬──────────────────────────┬─────────────────────────────────┬────────┐
│ year ┆ make          ┆ model                    ┆ VClass                          ┆ city08 │
│ ---  ┆ ---           ┆ ---                      ┆ ---                             ┆ ---    │
│ i16  ┆ cat           ┆ cat                      ┆ cat                             ┆ u8     │
╞══════╪═══════════════╪══════════════════════════╪═════════════════════════════════╪════════╡
│ 2000 ┆ Nissan        ┆ Altra EV                 ┆ Midsize Station Wagons          ┆ 81     │
│ 2000 ┆ Toyota        ┆ RAV4 EV                  ┆ Sport Utility Vehicle - 2WD     ┆ 81     │
│ 2001 ┆ Toyota        ┆ RAV4 EV                  ┆ Sport Utility Vehicle - 2WD     ┆ 81     │
│ 2001 ┆ Ford          ┆ Th!nk                    ┆ Two Seaters                     ┆ 74     │
│ 2001 ┆ Nissan        ┆ Hyper-Mini               ┆ Two Seaters                     ┆ 84     │
│ …    ┆ …             ┆ …        

In [23]:
print(autos
      .filter(standardize(pl.col('city08')).over(['year', 'make']) > 3)
      .select(['year', 'make', 'model', 'VClass', 'city08'])
     )

shape: (792, 5)
┌──────┬───────────────┬───────────────┬──────────────────────┬────────┐
│ year ┆ make          ┆ model         ┆ VClass               ┆ city08 │
│ ---  ┆ ---           ┆ ---           ┆ ---                  ┆ ---    │
│ i16  ┆ cat           ┆ cat           ┆ cat                  ┆ u8     │
╞══════╪═══════════════╪═══════════════╪══════════════════════╪════════╡
│ 1985 ┆ Ferrari       ┆ Testarossa    ┆ Two Seaters          ┆ 9      │
│ 1993 ┆ CX Automotive ┆ XM v6         ┆ Midsize Cars         ┆ 13     │
│ 1993 ┆ CX Automotive ┆ XM v6a        ┆ Midsize Cars         ┆ 13     │
│ 1993 ┆ Lexus         ┆ LS 400        ┆ Midsize Cars         ┆ 16     │
│ 1993 ┆ CX Automotive ┆ XM v6 Break   ┆ Small Station Wagons ┆ 13     │
│ …    ┆ …             ┆ …             ┆ …                    ┆ …      │
│ 1993 ┆ Lexus         ┆ SC 300/SC 400 ┆ Subcompact Cars      ┆ 16     │
│ 1993 ┆ Mitsubishi    ┆ Mirage        ┆ Subcompact Cars      ┆ 28     │
│ 1993 ┆ Plymouth      ┆ Colt      

### More Filtering with Window Expressions

In [26]:
print(autos
      .with_columns(model_age=(pl.col('year').max() - pl.col('year').min())
                    .over('model'))
     )

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬─────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ model_age │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ i16       │
│      ┆            ┆              ┆       ┆   ┆ ]           ┆ str         ┆           ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 1         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 8         │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00    ┆         

In [27]:
print(autos
      .with_columns(model_age=(pl.col('year').max() - pl.col('year').min())
                    .over('model'))
      .filter(pl.col('model_age') > 10)
     )

shape: (19_207, 16)
┌──────┬────────┬────────────┬───────┬───┬──────────────┬──────────────┬───────────┬───────────┐
│ year ┆ make   ┆ model      ┆ displ ┆ … ┆ createdOn    ┆ is_automatic ┆ num_gears ┆ model_age │
│ ---  ┆ ---    ┆ ---        ┆ ---   ┆   ┆ ---          ┆ ---          ┆ ---       ┆ ---       │
│ i16  ┆ cat    ┆ cat        ┆ f32   ┆   ┆ datetime[μs] ┆ str          ┆ u8        ┆ i16       │
╞══════╪════════╪════════════╪═══════╪═══╪══════════════╪══════════════╪═══════════╪═══════════╡
│ 1985 ┆ Dodge  ┆ Charger    ┆ 2.2   ┆ … ┆ 2013-01-01   ┆ false        ┆ 5         ┆ 39        │
│      ┆        ┆            ┆       ┆   ┆ 00:00:00     ┆              ┆           ┆           │
│ 1993 ┆ Toyota ┆ Corolla    ┆ 1.6   ┆ … ┆ 2013-01-01   ┆ true         ┆ 3         ┆ 41        │
│      ┆        ┆            ┆       ┆   ┆ 00:00:00     ┆              ┆           ┆           │
│ 1993 ┆ Toyota ┆ Corolla    ┆ 1.6   ┆ … ┆ 2013-01-01   ┆ false        ┆ 5         ┆ 41        │
│      ┆  

### Manipulating Data with Conditional Expressions

In [29]:
print(autos
      .group_by('fuelType')
      .len()
     )

shape: (15, 2)
┌─────────────────────────────┬───────┐
│ fuelType                    ┆ len   │
│ ---                         ┆ ---   │
│ cat                         ┆ u32   │
╞═════════════════════════════╪═══════╡
│ Gasoline or natural gas     ┆ 20    │
│ Midgrade                    ┆ 166   │
│ Regular Gas and Electricity ┆ 116   │
│ Hydrogen                    ┆ 32    │
│ Regular Gas or Electricity  ┆ 4     │
│ …                           ┆ …     │
│ Diesel                      ┆ 1284  │
│ Regular                     ┆ 29131 │
│ Gasoline or E85             ┆ 1402  │
│ Premium Gas or Electricity  ┆ 55    │
│ Gasoline or propane         ┆ 8     │
└─────────────────────────────┴───────┘


In [31]:
print(autos
      .select(pl.col('fuelType').value_counts(sort=True))
     )

shape: (15, 1)
┌─────────────────────────────────┐
│ fuelType                        │
│ ---                             │
│ struct[2]                       │
╞═════════════════════════════════╡
│ {"Regular",29131}               │
│ {"Premium",14699}               │
│ {"Gasoline or E85",1402}        │
│ {"Diesel",1284}                 │
│ {"Electricity",930}             │
│ …                               │
│ {"Premium Gas or Electricity",… │
│ {"Hydrogen",32}                 │
│ {"Gasoline or natural gas",20}  │
│ {"Gasoline or propane",8}       │
│ {"Regular Gas or Electricity",… │
└─────────────────────────────────┘


In [32]:
print(autos
      .select(pl.col('fuelType').value_counts(sort=True))
      .unnest('fuelType')
     )

shape: (15, 2)
┌────────────────────────────┬───────┐
│ fuelType                   ┆ count │
│ ---                        ┆ ---   │
│ cat                        ┆ u32   │
╞════════════════════════════╪═══════╡
│ Regular                    ┆ 29131 │
│ Premium                    ┆ 14699 │
│ Gasoline or E85            ┆ 1402  │
│ Diesel                     ┆ 1284  │
│ Electricity                ┆ 930   │
│ …                          ┆ …     │
│ Premium Gas or Electricity ┆ 55    │
│ Hydrogen                   ┆ 32    │
│ Gasoline or natural gas    ┆ 20    │
│ Gasoline or propane        ┆ 8     │
│ Regular Gas or Electricity ┆ 4     │
└────────────────────────────┴───────┘


In [35]:
print(autos
      .select(pl.col('fuelType')
              .value_counts(sort=True)
              .struct[0])
     )

shape: (15, 1)
┌────────────────────────────┐
│ fuelType                   │
│ ---                        │
│ cat                        │
╞════════════════════════════╡
│ Regular                    │
│ Premium                    │
│ Gasoline or E85            │
│ Diesel                     │
│ Electricity                │
│ …                          │
│ Premium Gas or Electricity │
│ Hydrogen                   │
│ Gasoline or natural gas    │
│ Gasoline or propane        │
│ Regular Gas or Electricity │
└────────────────────────────┘


##### Building `fuelTypeSimple` Expression

```sh
if 'Diesel' in fuel_type:
    fuel_type_simple = 'Diesel'
elif 'CNG' in fuel_type:
    fuel_type_simple = 'CNG'
elif 'Electricity' in fuel_type:
    fuel_type_simple = 'Electric'
else:
    fuel_type_simple = 'Gasoline'

##### Polar's Format

```sh
(pl.when(COND_EXPR)
   .then(EXPR)
 .when(COND_EXPR2)
   .then(EXPR2)
 ... 
 .otherwise(EXPRN)

In [36]:
fuel_str = pl.col('fuelType').cast(pl.String)

In [37]:
simple = (pl.when(fuel_str.str.contains('Diesel')).then(pl.lit('Diesel'))
          .when(fuel_str.str.contains('CNG')).then(pl.lit('CNG'))
          .when(fuel_str.str.contains('Electricity')).then(pl.lit('Electric'))
          .otherwise(pl.lit('Gasoline')))

In [38]:
print(autos
      .with_columns(simple.alias('fuelTypeSimple'))
     )

shape: (48_231, 16)
┌──────┬────────────┬─────────────┬───────┬───┬─────────────┬─────────────┬───────────┬────────────┐
│ year ┆ make       ┆ model       ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ fuelTypeSi │
│ ---  ┆ ---        ┆ ---         ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ mple       │
│ i16  ┆ cat        ┆ cat         ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ ---        │
│      ┆            ┆             ┆       ┆   ┆ ]           ┆ str         ┆           ┆ str        │
╞══════╪════════════╪═════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider      ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ Gasoline   │
│      ┆            ┆ Veloce 2000 ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆            │
│ 1985 ┆ Ferrari    ┆ Testarossa  ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ Gasoline   │
│      ┆            ┆             ┆       ┆   ┆ 00:00:00    ┆          

In [39]:
# Looking at CNG Column, as follows:
fuel_str = pl.col('fuelType').cast(pl.String)

In [40]:
simple = (pl.when(fuel_str.str.contains('Diesel')).then(pl.lit('Diesel'))
          .when(fuel_str.str.contains('CNG')).then(pl.lit('CNG'))
          .when(fuel_str.str.contains('Electricity')).then(pl.lit('Electric'))
          .otherwise(pl.lit('Gasoline')))

In [41]:
print(autos
      .with_columns(fuelTypeSimple=simple)
      .filter(pl.col('fuelTypeSimple') == 'CNG')
     )

shape: (60, 16)
┌──────┬─────────────┬─────────────┬───────┬───┬─────────────┬────────────┬───────────┬────────────┐
│ year ┆ make        ┆ model       ┆ displ ┆ … ┆ createdOn   ┆ is_automat ┆ num_gears ┆ fuelTypeSi │
│ ---  ┆ ---         ┆ ---         ┆ ---   ┆   ┆ ---         ┆ ic         ┆ ---       ┆ mple       │
│ i16  ┆ cat         ┆ cat         ┆ f32   ┆   ┆ datetime[μs ┆ ---        ┆ u8        ┆ ---        │
│      ┆             ┆             ┆       ┆   ┆ ]           ┆ str        ┆           ┆ str        │
╞══════╪═════════════╪═════════════╪═══════╪═══╪═════════════╪════════════╪═══════════╪════════════╡
│ 1996 ┆ Ford        ┆ Crown       ┆ 4.6   ┆ … ┆ 2013-01-01  ┆ true       ┆ 4         ┆ CNG        │
│      ┆             ┆ Victoria    ┆       ┆   ┆ 00:00:00    ┆            ┆           ┆            │
│      ┆             ┆ CNG         ┆       ┆   ┆             ┆            ┆           ┆            │
│ 1997 ┆ Ford        ┆ Crown       ┆ 4.6   ┆ … ┆ 2013-01-01  ┆ true       ┆

In [42]:
top_n = (pl.col('make')
         .value_counts(sort=True)
         .struct[0]
         .head(10)
        )

In [44]:
top_expr = (pl.when(pl.col('make').is_in(top_n))
            .then(pl.col('make'))
            .otherwise(pl.lit('Other'))
           )

In [46]:
print(autos
      .with_columns(simple_make=top_expr)
     )

shape: (48_231, 16)
┌──────┬────────────┬─────────────┬───────┬───┬─────────────┬─────────────┬───────────┬────────────┐
│ year ┆ make       ┆ model       ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ simple_mak │
│ ---  ┆ ---        ┆ ---         ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ e          │
│ i16  ┆ cat        ┆ cat         ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ ---        │
│      ┆            ┆             ┆       ┆   ┆ ]           ┆ str         ┆           ┆ cat        │
╞══════╪════════════╪═════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider      ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ Other      │
│      ┆            ┆ Veloce 2000 ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆            │
│ 1985 ┆ Ferrari    ┆ Testarossa  ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ Other      │
│      ┆            ┆             ┆       ┆   ┆ 00:00:00    ┆          

/var/folders/dk/f9pbhbp52qxc0613mpbtx8lm0000gn/T/ipykernel_5079/2555120211.py:2: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .with_columns(simple_make=top_expr)


In [47]:
# Limit Helper Function
def limit(col_name, n=10, default='other'):
    col = pl.col(col_name)
    top_n = (col
             .value_counts(sort=True)
             .struct[0]
             .head(n)
            )
    return (pl.when(col.is_in(top_n))
            .then(col)
            .otherwise(pl.lit(default))
           )

In [48]:
print(autos
      .with_columns(simple_make=limit('make'))
     )

shape: (48_231, 16)
┌──────┬────────────┬─────────────┬───────┬───┬─────────────┬─────────────┬───────────┬────────────┐
│ year ┆ make       ┆ model       ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ simple_mak │
│ ---  ┆ ---        ┆ ---         ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ e          │
│ i16  ┆ cat        ┆ cat         ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ ---        │
│      ┆            ┆             ┆       ┆   ┆ ]           ┆ str         ┆           ┆ cat        │
╞══════╪════════════╪═════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider      ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ other      │
│      ┆            ┆ Veloce 2000 ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆            │
│ 1985 ┆ Ferrari    ┆ Testarossa  ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ other      │
│      ┆            ┆             ┆       ┆   ┆ 00:00:00    ┆          

/var/folders/dk/f9pbhbp52qxc0613mpbtx8lm0000gn/T/ipykernel_5079/2798648594.py:2: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .with_columns(simple_make=limit('make'))


In [49]:
with pl.StringCache():
    autos2 = tweak_auto(raw)
    print(autos2
          .select(simple_make=limit('make', 20, 'other')))

shape: (48_231, 1)
┌─────────────┐
│ simple_make │
│ ---         │
│ cat         │
╞═════════════╡
│ other       │
│ other       │
│ Dodge       │
│ Dodge       │
│ Subaru      │
│ …           │
│ Subaru      │
│ Subaru      │
│ Subaru      │
│ Subaru      │
│ Subaru      │
└─────────────┘


### Handling Missing Data

In [50]:
print(autos
      .select(pl.col('*').is_null())
     )

shape: (48_231, 15)
┌───────┬───────┬───────┬───────┬───┬───────────┬───────────┬──────────────┬───────────┐
│ year  ┆ make  ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn ┆ is_automatic ┆ num_gears │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---          ┆ ---       │
│ bool  ┆ bool  ┆ bool  ┆ bool  ┆   ┆ bool      ┆ bool      ┆ bool         ┆ bool      │
╞═══════╪═══════╪═══════╪═══════╪═══╪═══════════╪═══════════╪══════════════╪═══════════╡
│ false ┆ false ┆ false ┆ false ┆ … ┆ false     ┆ false     ┆ false        ┆ false     │
│ false ┆ false ┆ false ┆ false ┆ … ┆ false     ┆ false     ┆ false        ┆ false     │
│ false ┆ false ┆ false ┆ false ┆ … ┆ false     ┆ false     ┆ false        ┆ false     │
│ false ┆ false ┆ false ┆ false ┆ … ┆ false     ┆ false     ┆ false        ┆ false     │
│ false ┆ false ┆ false ┆ false ┆ … ┆ false     ┆ false     ┆ false        ┆ false     │
│ …     ┆ …     ┆ …     ┆ …     ┆ … ┆ …         ┆ …         ┆ …            ┆ …         │
│

In [51]:
int(True)

1

In [52]:
print(autos
      .select(pl.all().is_null().sum())
     )

shape: (1, 15)
┌──────┬──────┬───────┬───────┬───┬───────────┬───────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---          ┆ ---       │
│ u32  ┆ u32  ┆ u32   ┆ u32   ┆   ┆ u32       ┆ u32       ┆ u32          ┆ u32       │
╞══════╪══════╪═══════╪═══════╪═══╪═══════════╪═══════════╪══════════════╪═══════════╡
│ 0    ┆ 0    ┆ 0     ┆ 963   ┆ … ┆ 0         ┆ 0         ┆ 0            ┆ 0         │
└──────┴──────┴───────┴───────┴───┴───────────┴───────────┴──────────────┴───────────┘


In [53]:
# Count of Null Values
print(autos
      .select(pl.all().null_count())
     )

shape: (1, 15)
┌──────┬──────┬───────┬───────┬───┬───────────┬───────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---          ┆ ---       │
│ u32  ┆ u32  ┆ u32   ┆ u32   ┆   ┆ u32       ┆ u32       ┆ u32          ┆ u32       │
╞══════╪══════╪═══════╪═══════╪═══╪═══════════╪═══════════╪══════════════╪═══════════╡
│ 0    ┆ 0    ┆ 0     ┆ 963   ┆ … ┆ 0         ┆ 0         ┆ 0            ┆ 0         │
└──────┴──────┴───────┴───────┴───┴───────────┴───────────┴──────────────┴───────────┘


In [54]:
print(autos
      .null_count()
     )

shape: (1, 15)
┌──────┬──────┬───────┬───────┬───┬───────────┬───────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---          ┆ ---       │
│ u32  ┆ u32  ┆ u32   ┆ u32   ┆   ┆ u32       ┆ u32       ┆ u32          ┆ u32       │
╞══════╪══════╪═══════╪═══════╪═══╪═══════════╪═══════════╪══════════════╪═══════════╡
│ 0    ┆ 0    ┆ 0     ┆ 963   ┆ … ┆ 0         ┆ 0         ┆ 0            ┆ 0         │
└──────┴──────┴───────┴───────┴───┴───────────┴───────────┴──────────────┴───────────┘


In [55]:
print(autos
      .describe()
     )

shape: (9, 16)
┌────────────┬────────────┬───────┬───────┬───┬───────────┬──────────────┬─────────────┬───────────┐
│ statistic  ┆ year       ┆ make  ┆ model ┆ … ┆ highway08 ┆ createdOn    ┆ is_automati ┆ num_gears │
│ ---        ┆ ---        ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---          ┆ c           ┆ ---       │
│ str        ┆ f64        ┆ str   ┆ str   ┆   ┆ f64       ┆ str          ┆ ---         ┆ f64       │
│            ┆            ┆       ┆       ┆   ┆           ┆              ┆ str         ┆           │
╞════════════╪════════════╪═══════╪═══════╪═══╪═══════════╪══════════════╪═════════════╪═══════════╡
│ count      ┆ 48231.0    ┆ 48231 ┆ 48231 ┆ … ┆ 48231.0   ┆ 48231        ┆ 48231       ┆ 48231.0   │
│ null_count ┆ 0.0        ┆ 0     ┆ 0     ┆ … ┆ 0.0       ┆ 0            ┆ 0           ┆ 0.0       │
│ mean       ┆ 2004.60571 ┆ null  ┆ null  ┆ … ┆ 25.822002 ┆ 2014-11-16   ┆ null        ┆ 5.347805  │
│            ┆            ┆       ┆       ┆   ┆           ┆ 10:20:26.646 ┆  

In [56]:
# Calculating % Method
print(autos
      .select(pl.all().is_null().mean() * 100)
     )

shape: (1, 15)
┌──────┬──────┬───────┬──────────┬───┬───────────┬───────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ    ┆ … ┆ highway08 ┆ createdOn ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---          ┆ ---       │
│ f64  ┆ f64  ┆ f64   ┆ f64      ┆   ┆ f64       ┆ f64       ┆ f64          ┆ f64       │
╞══════╪══════╪═══════╪══════════╪═══╪═══════════╪═══════════╪══════════════╪═══════════╡
│ 0.0  ┆ 0.0  ┆ 0.0   ┆ 1.996641 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0          ┆ 0.0       │
└──────┴──────┴───────┴──────────┴───┴───────────┴───────────┴──────────────┴───────────┘


In [57]:
# Checking for Null Values
print(autos
      .select(pl.all().is_null().cast(pl.Int32).mean() * 100)
      .pipe(lambda df_: df_.select([col.name 
                                    for col in df_.select(pl.col(pl.Float64)> 0) 
                                    if col.all()]))
     )

shape: (1, 4)
┌──────────┬───────────┬──────────┬──────────┐
│ displ    ┆ cylinders ┆ trany    ┆ drive    │
│ ---      ┆ ---       ┆ ---      ┆ ---      │
│ f64      ┆ f64       ┆ f64      ┆ f64      │
╞══════════╪═══════════╪══════════╪══════════╡
│ 1.996641 ┆ 2.000788  ┆ 0.022807 ┆ 2.458999 │
└──────────┴───────────┴──────────┴──────────┘


In [58]:
#  Checking for NaN Values
print(autos
      .select(cs.numeric().is_nan().cast(pl.Int32).mean() * 100)
      .pipe(lambda df_: df_.select([col.name 
                                    for col in df_.select(pl.col(pl.Float64)> 0) 
                                    if col.all()]))
     )

shape: (0, 0)
┌┐
╞╡
└┘


### Dropping Rows with Missing Values

In [61]:
# Placing Model
pca = decomposition.PCA()

In [62]:
pca.fit(autos.select(cs.numeric()))

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [63]:
# Going Nuclear
print(autos
      .drop_nulls()
     )

shape: (46_086, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [65]:
print(autos
      .drop_nulls(subset=(['cylinders', 'displ']))
     )

shape: (47_266, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [66]:
print(autos
      .drop_nulls(subset=cs.numeric())
     )

shape: (47_266, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [67]:
# Running again with PCA() Model
pca = decomposition.PCA()

In [68]:
# Fitting Model
pca.fit(autos.select(cs.numeric()).drop_nulls())

PCA()

In [70]:
(autos
 .filter(pl.col('num_gears', 'is_automatic').is_null())
)

ComputeError: The predicate passed to 'LazyFrame.filter' expanded to multiple expressions: 

	col("num_gears").is_null(),
	col("is_automatic").is_null(),
This is ambiguous. Try to combine the predicates with the 'all' or `any' expression.

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'filter' <---
DF ["year", "make", "model", "displ"]; PROJECT */15 COLUMNS; SELECTION: None

In [71]:
print(autos
      .filter(pl.col('num_gears').is_null() & pl.col('is_automatic').is_null())
     )

shape: (0, 15)
┌──────┬──────┬───────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat  ┆ cat   ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪══════╪═══════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
└──────┴──────┴───────┴───────┴───┴───────────┴──────────────┴──────────────┴───────────┘


In [72]:
# All Horizontal Function on Row-wise
print(autos
      .filter(pl.all_horizontal(
          pl.col('num_gears', 'is_automatic').is_null()))
     )

shape: (0, 15)
┌──────┬──────┬───────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make ┆ model ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---  ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat  ┆ cat   ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪══════╪═══════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
└──────┴──────┴───────┴───────┴───┴───────────┴──────────────┴──────────────┴───────────┘


In [74]:
# Applying Filter Inversion
print(autos
      .filter(~pl.all_horizontal(
          pl.col('num_gears', 'is_automatic').is_null()))
     )

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

### Filling in Missing Values